<a href="https://colab.research.google.com/github/Ibrahim-Ezmazy/python-Projects/blob/main/CNN_CoffeeDiseases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
# Mount Google Drive if needed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Define paths to training and validation data
train_dir = '/content/drive/MyDrive/data/train'
validation_dir = '/content/drive/MyDrive/data/valid'

In [ ]:
# Data Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')



Found 539 images belonging to 2 classes.
Found 539 images belonging to 2 classes.


In [ ]:
# Build CNN Model

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


In [ ]:
# Compile Model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train Model
# Train Model
history = model.fit(
      train_generator,
      epochs=10,
      steps_per_epoch=len(train_generator),
      validation_data=validation_generator,
      validation_steps=len(validation_generator))



Epoch 1/10
27/27 [==============================] - 230s 8s/step - loss: 0.5319 - accuracy: 0.7032 - val_loss: 0.3407 - val_accuracy: 0.8627
Epoch 2/10
27/27 [==============================] - 143s 5s/step - loss: 0.3386 - accuracy: 0.8701 - val_loss: 0.2881 - val_accuracy: 0.8905
Epoch 3/10
27/27 [==============================] - 152s 6s/step - loss: 0.3038 - accuracy: 0.8961 - val_loss: 0.2018 - val_accuracy: 0.9314
Epoch 4/10
27/27 [==============================] - 146s 5s/step - loss: 0.2306 - accuracy: 0.9072 - val_loss: 0.1515 - val_accuracy: 0.9406
Epoch 5/10
27/27 [==============================] - 164s 6s/step - loss: 0.1325 - accuracy: 0.9481 - val_loss: 0.0856 - val_accuracy: 0.9685
Epoch 6/10
27/27 [==============================] - 165s 6s/step - loss: 0.1080 - accuracy: 0.9685 - val_loss: 0.1109 - val_accuracy: 0.9536
Epoch 7/10
27/27 [==============================] - 151s 6s/step - loss: 0.1938 - accuracy: 0.9202 - val_loss: 0.1647 - val_accuracy: 0.9314
Epoch 8/10
27

In [ ]:
# Evaluate Model
loss, accuracy = model.evaluate(validation_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


27/27 [==============================] - 63s 2s/step - loss: 0.2317 - accuracy: 0.8868
Validation Loss: 0.23169580101966858
Validation Accuracy: 0.8868274688720703


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Predict on validation data
y_pred = model.predict(validation_generator).flatten() > 0.5  # Assuming binary classification

# True labels
y_true = validation_generator.classes

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# True Positives, False Positives, True Negatives, False Negatives
TN, FP, FN, TP = conf_matrix.ravel()

# Precision, Recall, F-score
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f_score = f1_score(y_true, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print("\nPrecision:", precision)
print("Recall:", recall)
print("F-score:", f_score)


27/27 [==============================] - 66s 2s/step
Confusion Matrix:
[[142 112]
 [173 112]]

Precision: 0.5
Recall: 0.3929824561403509
F-score: 0.4400785854616896


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
# **RESNET50**

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [15]:

# Load dataset
train_data_dir = '/content/drive/MyDrive/dataset/train'
test_data_dir = '/content/drive/MyDrive/dataset/test'

img_width, img_height = 155, 155
batch_size = 32


In [16]:

# Data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 511 images belonging to 2 classes.
Found 167 images belonging to 2 classes.


In [17]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [18]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [19]:

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)


Epoch 1/10
15/15 [==============================] - 248s 16s/step - loss: 0.7480 - accuracy: 0.5637 - val_loss: 0.6853 - val_accuracy: 0.4875
Epoch 2/10
15/15 [==============================] - 113s 7s/step - loss: 0.6722 - accuracy: 0.5771 - val_loss: 0.5969 - val_accuracy: 0.8687
Epoch 3/10
15/15 [==============================] - 113s 7s/step - loss: 0.6349 - accuracy: 0.6180 - val_loss: 0.5560 - val_accuracy: 0.8500
Epoch 4/10
15/15 [==============================] - 126s 8s/step - loss: 0.5908 - accuracy: 0.7704 - val_loss: 0.5165 - val_accuracy: 0.8062
Epoch 5/10
15/15 [==============================] - 118s 8s/step - loss: 0.5783 - accuracy: 0.7349 - val_loss: 0.5033 - val_accuracy: 0.8562
Epoch 6/10
15/15 [==============================] - 113s 8s/step - loss: 0.5681 - accuracy: 0.7328 - val_loss: 0.4740 - val_accuracy: 0.7750
Epoch 7/10
15/15 [==============================] - 127s 8s/step - loss: 0.5618 - accuracy: 0.6848 - val_loss: 0.4141 - val_accuracy: 0.8813
Epoch 8/10
1

In [20]:


# Evaluate the model
score = model.evaluate(validation_generator)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])


6/6 [==============================] - 27s 4s/step - loss: 0.3885 - accuracy: 0.9042
Test Loss: 0.3884824514389038
Test Accuracy: 0.9041916131973267


In [27]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Predict on validation data
y_pred = model.predict(validation_generator).flatten() > 0.5  # Assuming binary classification

# True labels
y_true = validation_generator.classes

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# True Positives, False Positives, True Negatives, False Negatives
TN, FP, FN, TP = conf_matrix.ravel()

# Precision, Recall, F-score
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f_score = f1_score(y_true, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print("\nPrecision:", precision)
print("Recall:", recall)
print("F-score:", f_score)


6/6 [==============================] - 35s 5s/step
Confusion Matrix:
[[53 33]
 [43 38]]

Precision: 0.5352112676056338
Recall: 0.4691358024691358
F-score: 0.4999999999999999
